<a href="https://colab.research.google.com/github/HWKim-postech/PlayGround/blob/main/Basic_FC_and_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Basic Fully Connected and CNN model built by HW Kim

I wrote the simplest, but the most important models of ML. 
If you are very beginning of ML, it can be helpful.

If you find errors (Including tiny mistakes), please let me know!

And personally, the best way to learn something is try to find the answer on yorself. 
This is simle model but I'm sure that there're some confusing concepts. 

**Everytime you find them, google it and try to understand it.** 
**Not just pass it and think "I know it"**

After finishing hard works, you can feel the improvements!

(Any questions: kimhyunwoo554@gmail.com)

#Download the libraries for models

If you use Anaconda or other python platforms, you might not need to download it.

But it you use simple VS CODE, then you need to downlaod these libraries on CMD

*Go to the cmd and open it as Admin mode. And type it : "pip install (What you want to install)"

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import argparse

##Next, I'll declare constants that will be used frequently.

And I also declare the device that I use.

In [74]:
parser = argparse.ArgumentParser()

parser.add_argument("--n_epochs", type = int, default = 10, help = "# of epochs during training")
parser.add_argument("--n_BatchSize", type = int, default = 32, help = "Size of Batches")
parser.add_argument("--lr", type = float, default = 0.003, help = "learning rate")
parser.add_argument("--betas", type = float, default = (0.9, 0.999), help = "Constants for the Adam")
parser.add_argument("--img_size", type = int, default = 32, help = "Image size")
parser.add_argument("--interval", type = int, default = 100, help = "Check Point interval")

opt = parser.parse_args(args=[])
device = torch.device('cpu')
'''
  if torch.cuda.is_available():
    device = torch.device('cuda')
  else:
    device = torch.device('cpu')
'''

##Now we're gonna download the Dataset from the MNIST (Hand-Written digits) 
http://yann.lecun.com/exdb/mnist/

In [3]:
train_data = datasets.MNIST(root = "../MNIST", download=True, train = True, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))
test_data = datasets.MNIST(root = "../MNIST", download=True, train = False, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))

train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=opt.n_BatchSize)
test_loader = torch.utils.data.DataLoader(test_data, shuffle = False, batch_size=opt.n_BatchSize)


##This is CIFAR10 dataset

https://www.cs.toronto.edu/~kriz/cifar.html

In [50]:
train_data = datasets.CIFAR10(root = "../CIFAR", download=True, train = True, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))
test_data = datasets.CIFAR10(root = "../CIFAR", download=True, train = False, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))

train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=opt.n_BatchSize)
test_loader = torch.utils.data.DataLoader(test_data, shuffle = False, batch_size=opt.n_BatchSize)

Extracting ../CIFAR/cifar-10-python.tar.gz to ../CIFAR
Files already downloaded and verified


##It is very important to check the input data.

And Tensor is quite confusing concept. So you need to study about it.

It can lead us to the deeper understanding.
.

Read the following website:

https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

In [118]:
for img, label in train_loader:
  print("Type of image:",img[0].type(), "\nSize of image:",img[0].size())
  print("Type of label:",label[0].type(), "\nValue in label[0]:",label[0].item())
  break

Type of image: torch.FloatTensor 
Size of image: torch.Size([3, 32, 32])
Type of label: torch.LongTensor 
Value in label[0]: 4


##Let's build the model (FC layer)

I use Batch Normalization(BN) and Droput to introduce the concepts (You have to read documents about BN, Dropout)

In this case, we don't need to use BN, Droput techniques. We can get enough performance from a simple FC model.

But it can prevent the overfitting, spikes.

In [32]:
Yclass FC(nn.Module):
  def __init__(self):
    super(FC, self).__init__()
    self.fc1 = nn.Linear(opt.img_size**2, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)
    self.BN1 = nn.BatchNorm1d(256)
    self.BN2 = nn.BatchNorm1d(128)

  #You can add FC, BN and dropout layer. Let's adjust it and test your own model!
  def forward(self, x):
    x = x.view(-1, opt.img_size**2)
    x = self.fc1(x)
    x = self.BN1(x)
    x = F.relu(x)
    x = F.dropout(x, p = 0.5)
    x = self.fc2(x)
    x = self.BN2(x)
    x = F.relu(x)
    x = F.dropout(x, p = 0.5)
    x = self.fc3(x)
    x = F.log_softmax(x)
  
    return x

##Let's build the model(CNN)

When building the CNN, we have to consider about the dimensionality more carefully.

If you download the MNIST, self,conv1 has input channel as 1 (gray)

If CIFAR10, it is 3 (RGB)

I wrote CNN model based on CIFAR10. So if you want to use MNIST dataset, then 

you need to tune the parameter. It's gonna be good study.

*(MNIST = [1, 28, 28], CIFAR10 = [3, 32, 32])*

In [114]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = nn.Linear(8*8*16, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 10)

    self.BN_CNN1 = nn.BatchNorm2d(8)


    self.BN_FC1 = nn.BatchNorm1d(64)
    self.BN_FC2 = nn.BatchNorm1d(32)

  #You can add Convolution layer, FC, BN, dropout and pooling layer. Let's adjust it and test your own model!
  def forward(self, x):
    x = self.conv1(x)
    x = self.BN_CNN1(x)
    x = F.relu(x)
    x = F.dropout2d(x, p = 0.5)
    x = self.pool(x)

    x = self.conv2(x)
    x = self.pool(x)
    x = F.relu(x)

    x = x.view(-1, 8*8*16)

    x = self.fc1(x)
    x = self.BN_FC1(x)
    x = F.relu(x)
    x = F.dropout(x, p = 0.5)

    x = self.fc2(x)
    x = self.BN_FC2(x)
    x = F.relu(x)
    x = F.dropout(x, p = 0.5)

    x = self.fc3(x)
    x = F.log_softmax(x)

    return x

##Since our model operated on device(GPU or CPU), let's allocate FC(or CNN) to the device.

Also declare the optimizer, loss function

In [115]:
model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = opt.lr, betas = opt.betas)
criterion = nn.CrossEntropyLoss()

##Define the evalulate section

In [116]:
def eval(model, test_loader):
  model.eval()
  eval_loss = 0
  acc = 0
  with torch.no_grad():
    for img, label in test_loader:
      img = img.to(device)
      label = label.to(device)
      output = model(img)
      eval_loss += criterion(output, label).item()
      pred = output.max(1, keepdim = True)[1]
      acc += pred.eq(label.view_as(pred)).sum().item() #I refer to "파이썬 딥러닝 파이토치 by 이경택, 방성수, 안상준; ISBN : 978-89-5674-857-3" to write accuracy calculating codes.
      
  return eval_loss/len(test_loader.dataset), 100*acc/len(test_loader.dataset)

##LET'S RUN IT!

In [ ]:
for epochs in range(1, opt.n_epochs+1):
  model.train()
  for batch_idx, (img, label) in enumerate(train_loader):
    img = img.to(device)
    label = label.to(device)
    output = model(img)
    optimizer.zero_grad()
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()

    if batch_idx % opt.interval == 0 :
      print("[Epochs : %d] [batch_index = %d] [loss = %f]"%(epochs, batch_idx, loss.item()))

  Evaluate_loss, acc = eval(model, test_loader)
  print("Evaluate Loss =", Evaluate_loss, "\tAccuracy(%) =", acc)